<a href="https://colab.research.google.com/github/jhalek/project_one_team_six/blob/jeremy_branch/Copy_of_Project_1_(crime_%26_real_estate).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import dependencies
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import csv
from pprint import pprint
import json
from google.colab import files
uploaded = files.upload()

Saving Elyria_Police_Department.csv to Elyria_Police_Department.csv
Saving Lakewood_Police_Department.csv to Lakewood_Police_Department.csv
Saving median_home_data.csv to median_home_data.csv


In [0]:
#import elyria crime data
file_1 = "Elyria_Police_Department.csv"
el_crime_data = pd.read_csv(file_1)

#import lakewood crime data
file_2 = "Lakewood_Police_Department.csv"
lw_crime_data = pd.read_csv(file_2)

FileNotFoundError: ignored

In [0]:
#recode zip +4 to regular zip and convert to float
el_crime_data['zip'] = el_crime_data['zip'].replace('44035-5622', '44305')
el_crime_data['zip'] = el_crime_data['zip'].replace('44035-2819', '44305')
el_crime_data['zip'] = el_crime_data['zip'].astype('float64') 

NameError: ignored

In [0]:
#remove unnecessary columns from lakewood data and create new dataframe
crime_data_merged = lw_crime_data.iloc[:,0:20]

#union the lakewood and elyria datasets
crime_data_merged = pd.concat([crime_data_merged,el_crime_data])

NameError: ignored

In [0]:
#create table with only needed columns and preview
crime_data_summary = crime_data_merged.iloc[:,[0,2,3,8,10,19]]

In [0]:
#extracts month and year from incident_datetime column
crime_data_summary['year'] = pd.DatetimeIndex(crime_data_summary['incident_datetime']).year
crime_data_summary['month'] = pd.DatetimeIndex(crime_data_summary['incident_datetime']).month

/anaconda3/envs/pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#remove unecessary zip codes and years using loc
crime_data_summary = crime_data_summary.loc[(crime_data_summary['zip']==44107) | (crime_data_summary['zip']==44035)]
crime_data_summary = crime_data_summary.loc[(crime_data_summary['year']<2019) & (crime_data_summary['year']>2009)]

In [0]:
#group incidents by month, year, zip
crime_data_grouped = crime_data_summary.groupby(['zip','month','year']).count().reset_index()

In [0]:
#import home data
file_3 = "median_home_data.csv"
home_data = pd.read_csv(file_3)

In [0]:
#remove unecessary zip codes using loc
home_data_summary = home_data.loc[(home_data['RegionName']==44107) | (home_data['RegionName']==44035)]

In [0]:
#transpose data frame
home_data_transp = home_data_summary.T.reset_index()

#rename columns
home_data_transp=home_data_transp.rename(index=str, columns={92: "Lakewood_Real_Estate_44107", 100: "Elyria_Real_Estate_44035"})
home_data_transp = home_data_transp.iloc[6:280,:]

home_data_transp.head()

,index,Lakewood_Real_Estate_44107,Elyria_Real_Estate_44035
6,1996_04,64,62
7,1996_05,64,62
8,1996_06,65,63
9,1996_07,65,62
10,1996_08,65,63


In [0]:
#parse index string for month and year
home_data_transp['month'] = home_data_transp['index'].str.split('_').str[1]
home_data_transp['year'] = home_data_transp['index'].str.split('_').str[0]
home_data_transp.iloc[:,[1,2,3,4]]
home_data_transp.head()

,index,Lakewood_Real_Estate_44107,Elyria_Real_Estate_44035,month,year
6,1996_04,64,62,04,1996
7,1996_05,64,62,05,1996
8,1996_06,65,63,06,1996
9,1996_07,65,62,07,1996
10,1996_08,65,63,08,1996


In [0]:
#remove columns for union / data reshaping
lakewood_stacked=home_data_transp.iloc[:,[1,3,4]]
elyria_stacked=home_data_transp.iloc[:,[2,3,4]]

#add zip as a column
lw_zip='44107'
el_zip='44035'
lakewood_stacked['zip']=lw_zip
elyria_stacked['zip']=el_zip

/anaconda3/envs/pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [0]:
#stack real estate data frames
home_prices= pd.concat([lakewood_stacked,elyria_stacked],sort=True)
home_prices=home_prices.reset_index()
home_prices=home_prices.iloc[:,[1,2,3,4,5]]

In [0]:
#recode NaN with zero and create new column for pricing
home_prices['Elyria_Real_Estate_44035'] = home_prices['Elyria_Real_Estate_44035'].replace('nan', np.nan).fillna(0)
home_prices['Lakewood_Real_Estate_44107'] = home_prices['Lakewood_Real_Estate_44107'].replace('nan', np.nan).fillna(0)
home_prices['sq_foot_prices']=home_prices['Lakewood_Real_Estate_44107']+home_prices['Elyria_Real_Estate_44035']


In [0]:
#reduce year paremeter to match crime data and reduce dataset size
home_prices['year'] = home_prices['year'].astype('float64') 
home_prices['month'] = home_prices['month'].astype('float64') 
home_prices['zip'] = home_prices['zip'].astype('float64') 
home_prices = home_prices.loc[(home_prices['year']<2019) & (home_prices['year']>2009)]
home_prices=home_prices.iloc[:,[2,3,4,5]]
home_prices.head()

,month,year,zip,sq_foot_prices
165,1.0,2010.0,44107.0,80.0
166,2.0,2010.0,44107.0,80.0
167,3.0,2010.0,44107.0,80.0
168,4.0,2010.0,44107.0,80.0
169,5.0,2010.0,44107.0,81.0


In [0]:
home_prices['zip'].value_counts()

44107.0    108
44035.0    108
Name: zip, dtype: int64

In [0]:
crime_data_grouped['zip'].value_counts()

44107.0    104
44035.0     59
Name: zip, dtype: int64

In [0]:
analysis_df = pd.merge(home_prices, crime_data_grouped,  how='inner', on=['zip','month','year'])
analysis_df.head()

,month,year,zip,sq_foot_prices,incident_id,incident_datetime,incident_type_primary,city,parent_incident_type
0,1.0,2010.0,44107.0,80.0,529,529,529,529,529
1,2.0,2010.0,44107.0,80.0,516,516,516,516,516
2,3.0,2010.0,44107.0,80.0,574,574,574,574,574
3,4.0,2010.0,44107.0,80.0,593,593,593,593,593
4,5.0,2010.0,44107.0,81.0,657,657,657,657,657


In [0]:
analysis_df.groupby(['zip']).describe().T

zip                              44035.0      44107.0
city                  count    59.000000   104.000000
                      mean    285.067797   665.211538
                      std     102.199748   147.944537
                      min       1.000000     1.000000
                      25%     260.500000   553.500000
                      50%     308.000000   671.000000
                      75%     344.500000   777.250000
                      max     444.000000   912.000000
incident_datetime     count    59.000000   104.000000
                      mean    285.067797   665.894231
                      std     102.199748   148.005374
                      min       1.000000     1.000000
                      25%     260.500000   553.500000
                      50%     308.000000   672.000000
                      75%     344.500000   778.000000
                      max     444.000000   914.000000
incident_id           count    59.000000   104.000000
                      mean    285.067797   665.894231
                      std     102.199748   148.005374
                      min       1.000000     1.000000
                      25%     260.500000   553.500000
                      50%     308.000000   672.000000
                      75%     344.500000   778.000000
                      max     444.000000   914.000000
incident_type_primary count    59.000000   104.000000
                      mean    285.067797   665.894231
                      std     102.199748   148.005374
                      min       1.000000     1.000000
                      25%     260.500000   553.500000
                      50%     308.000000   672.000000
...                                  ...          ...
month                 std       3.434704     3.450222
                      min       1.000000     1.000000
                      25%       3.500000     3.000000
                      50%       6.000000     6.000000
                      75%       9.000000     9.000000
                      max      12.000000    12.000000
parent_incident_type  count    59.000000   104.000000
                      mean    285.067797   665.894231
                      std     102.199748   148.005374
                      min       1.000000     1.000000
                      25%     260.500000   553.500000
                      50%     308.000000   672.000000
                      75%     344.500000   778.000000
                      max     444.000000   914.000000
sq_foot_prices        count    59.000000   104.000000
                      mean     59.203390    89.259615
                      std       2.265024    14.002770
                      min      56.000000    75.000000
                      25%      58.000000    78.000000
                      50%      59.000000    85.000000
                      75%      61.000000    95.500000
                      max      66.000000   126.000000
year                  count    59.000000   104.000000
                      mean   2012.881356  2013.846154
                      std       1.439405     2.518935
                      min    2010.000000  2010.000000
                      25%    2012.000000  2012.000000
                      50%    2013.000000  2014.000000
                      75%    2014.000000  2016.000000
                      max    2015.000000  2018.000000

[64 rows x 2 columns]

In [0]:
#creating new variable for category colors
analysis_df['type_colored'] = analysis_df['zip'].replace({44035.0:'lightskyblue', 44107.0:'lightcoral'})

# Build a scatter plot for temperature
plt.scatter(analysis_df["incident_type_primary"], analysis_df["sq_foot_prices"], c=analysis_df['type_colored'])

#Incorporate the other graph properties
plt.title("Crime by Home Value")
plt.ylabel("Home Value by Square Foot")
plt.xlabel("Reporting Crimes")
plt.grid(True)


# Show plot
plt.show()  

NameError: ignored